In [1]:




"""
Running algorithm to search best fitting model


Next steps:
    - Try meta features
    - Consistency check
    - Feature interaction / polynomial
    - Using era ?
    - hardcore EDA
"""



#=========================================================================================================
#================================ 0. MODULE


# Class
from numerai import Numerai

# Architecture
from architecture import models, nCores


#=========================================================================================================
#================================ 1. CLASS


stacking = Numerai(week=109)


#=========================================================================================================
#================================ 2. FEATURE ENGINEERING


# metafeature = ['variance', 'mean', 'distance']
# stages = [2, 0, 2]
# stacking.add_metafeature(metafeature, stages)


#=========================================================================================================
#================================ 3. TRAINING MODEL


###### MACHINE LEARNING
stacking.add_model(models)
stacking.fit_tune(nCores, stageNumber=1, neuralNetworkCompiler=True, evaluate=True, interaction=None)


---------------------------------------------
>> Loading data...done

Xtrain1: (157445, 50) 
Ytrain1: (157445,) 
Xtrain2: (158232, 50) 
Ytrain2: (158232,) 
Xtest: (77936, 50) 
Ytest: (77936,) 
Xvalid: (46362, 50) 
Yvalid: (46362,) 
Submit data: (243281, 50)


---------------------------------------------
>> Processing first stage

>> Processing ExtraTrees1 ------

Step 1...done in 0h 0m 22s
log loss : 0.6930368701955091

Step 2...done in 0h 0m 21s
log loss : 0.6930132622867041

Step 3...done in 0h 0m 21s
log loss : 0.6930529165701654


>> Processing ExtraTrees2 ------

Step 1...done in 0h 0m 19s
log loss : 0.6930387244985143

Step 2...done in 0h 0m 19s
log loss : 0.693052411614954

Step 3...done in 0h 0m 21s
log loss : 0.6929727032516171


First stage running time 0h 2m 5s


In [13]:
###### DEEP LEARNING
stacking.neuralNetworkCompiler(learningRate=0.0001, batch=64, epoch=10, cvNumber=2, displayStep=500, evaluate=True, useGPU=True)



---------------------------------------------
>> Processing compilation [Neural Network]


Loop number 1

>> Learning: 67 parameters

Epoch: 1, step:   500, training loss: 0.69414, validation loss: 0.69316
Epoch: 1, step:  1000, training loss: 0.69324, validation loss: 0.69341
Epoch: 1, step:  1500, training loss: 0.69333, validation loss: 0.69323
Epoch: 1, step:  2000, training loss: 0.69319, validation loss: 0.69316
Epoch: 1, step:  2500, training loss: 0.69318, validation loss: 0.69315
Epoch: 2, step:  3000, training loss: 0.69315, validation loss: 0.69317
Epoch: 2, step:  3500, training loss: 0.69318, validation loss: 0.69315
Epoch: 2, step:  4000, training loss: 0.69315, validation loss: 0.69315
Epoch: 2, step:  4500, training loss: 0.69317, validation loss: 0.69315
Epoch: 2, step:  5000, training loss: 0.69317, validation loss: 0.69315
Epoch: 2, step:  5500, training loss: 0.69317, validation loss: 0.69315
Epoch: 3, step:  6000, training loss: 0.69317, validation loss: 0.69316


In [ ]:
#=========================================================================================================
#================================ 4. PREDICTION

# stacking.fit_tune(nCores, neuralNetworkCompiler=False, evaluate=False, interaction=None)
# stacking.neuralNetwork(learningRate=0.0001, batch=64, epoch=2, cvNumber=2, displayStep=10000, evaluate=False, useGPU=True)
# stacking.submit(5, submissionNumber=2, week=109)

In [11]:
stacking.compilation_data[stacking.datasetToUse].shape[0]*3/4

177126.0

In [8]:
11000*64/4

176000.0